In [1]:
import pandas as pd
import numpy as np
import re, os

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
jet = plt.get_cmap('jet')


import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from statsmodels.regression.linear_model import WLS
from statsmodels.stats.weightstats import ttest_ind
from statsmodels.discrete.discrete_model import Probit
from scipy.stats import norm

from statsmodels.discrete.discrete_model import Poisson

import itertools
data_path = "C:/Users/SpiffyApple/Documents/USC/OwnResearch/proposal"

In [2]:
#convenience function
def toLatex(tmpDF, file_name):
    with open("/".join([data_path,'output',file_name]), 'w+') as f:
        f.write(tmpDF.to_latex())
    return(tmpDF)

def toExcel(tmpDF, file_name):
    tmpDF.to_excel("/".join([data_path,'output',file_name]))
    return(tmpDF)

# Data Processing

The data come from [here](https://www.census.gov/programs-surveys/ahs/data/2015/ahs-2015-public-use-file--puf-/2015-ahs-metropolitan-puf-microdata.html). Important AHS data documentation [here](https://www.census.gov/content/dam/Census/programs-surveys/ahs/tech-documentation/2015/Getting%20Started%20with%20the%20AHS%20PUF.pdf), variable [labels](https://www.census.gov/data-tools/demo/codebook/ahs/ahsdict.html?s_appName=ahsdict&s_year=2015)

Other years include:, [2017]()
[2013](https://www.census.gov/programs-surveys/ahs/data/2013/ahs-2013-public-use-file--puf-/ahs-2013-national-public-use-file--puf-.html), and 
[2011](https://www.census.gov/programs-surveys/ahs/data/2011/ahs-national-and-metropolitan-puf-microdata.html)

Another useful link is the [QuickFacts](https://www.census.gov/quickfacts/fact/table/milwaukeecitywisconsin/PST045217) for Milwaukee which gives basic information on housing and demographics

### AHS Data

In [3]:
dfm = pd.read_csv("/".join([data_path, 'ahs2015m.csv']))
dfn = pd.read_csv("/".join([data_path, '2015householdNational.csv']))
labels = pd.read_csv("/".join([data_path, 'ahs 2015 value labels.csv']), encoding='latin-1')
labels.drop(['Table','Type','FLAT','METRO',"FMTName"],axis=1, inplace=True)

In [4]:
#merge the national and metropolitan files
df = pd.concat([dfm.loc[:,dfm.columns.intersection(dfn.columns)],dfn]) 
df.replace("'",'',regex=True,inplace=True)
df.shape

C:\Users\SpiffyApple\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


(94379, 1091)

In [5]:
# Clean up the data
df.replace("'",'',regex=True,inplace=True) #remove "'" marks to convert to numerical
df = df.astype(np.float) #convert all to float - useful because it clears up the issue of entries such as '01'
df.replace(-9,np.nan, inplace=True) #replace -9s with np.nan

df.drop(df.columns[df.isnull().all()], axis=1, inplace=True) #drop columns that contain all NULLs

df.drop(df.columns[df.isnull().sum()/df.shape[0]>.9], axis=1, inplace=True) #drop columns in which more than 90% of the data are NULLs
df.shape

(94379, 1013)

### Add Labels to AHS Data values

Labels file comes from the Label Values file [here](https://www.census.gov/programs-surveys/ahs/data/2015/ahs-2015-public-use-file--puf-/ahs-2015-national-public-use-file--puf-.html). However, this file is only used to replace the values. 

I also downloaded the entire codebook from [AHS Codebook](https://www.census.gov/data-tools/demo/codebook/ahs/ahsdict.html) that details a lot more information than the label values has. It is also a convenient way to search for topics and the like

In [6]:
## upload labels data
labels = pd.read_csv("/".join([data_path, 'ahs 2015 value labels.csv']), encoding='latin-1')
labels.drop(['Table','Type','FLAT','METRO',"FMTName"],axis=1, inplace=True)

In [7]:
labels.Value.replace(".N|.M|N|M",-9,regex=True,inplace=True)

In [8]:
## pandas can only replace values in a structured manner through dictionaries
## to do that, I set values as indexes and convert the remaining Series
## into a dictionary for each column name
## there's the added complicationg of needing to compare indexes of same type

labels.set_index('Value',inplace=True)    

In [9]:
reNameDict = {}
for label, group in labels.groupby("NAME"):
    if label in df.columns:
        if np.issubdtype(np.object,df.loc[:,label].dtype ):
            reNameDict[label] = group.Label.to_dict()
        else:
            group.index = group.index.astype(df.loc[:,label].dtype)
            reNameDict[label] = group.Label.to_dict()

In [10]:
df.replace(reNameDict,inplace=True)

In [11]:
df.replace(-9,np.nan, inplace=True)
df.replace(-6,np.nan, inplace=True)

In [12]:
df.drop(df.columns[df.isnull().all()], axis=1, inplace=True) #drop columns that contain all NULLs

df.drop(df.columns[df.isnull().sum()/df.shape[0]>.9], axis=1, inplace=True) #drop columns in which more than 90% of the data are NULLs
df.shape

(94379, 1012)

In [13]:
## labels for columns
ahsdict = pd.read_csv("/".join([data_path, 'AHSDICT_28AUG18_23_35_13_43_S.csv']))
#drop unnecessary columns
ahsdict.drop(['Disclosure','Variable_Number','INUNIVERSE','Topic_Number','Present_in_Survey_Years','Topic_Subtopic_ID','Imputation_Strategy','Question','Instrument_Variable_Name','Survey Year','Subtopic'],axis=1,inplace=True)
ahsdict.set_index('Variable',inplace=True)

In [14]:
df.drop(df.columns[df.columns.str.lower().str.contains("wgt|weight")], axis=1,inplace=True) #drop weight cols
#df.drop("Unnamed: 0", axis=1, inplace=True) # drop unnamed col
df.drop(df.columns[df.isnull().all()], axis=1, inplace=True) #drop cols that are all null

In [15]:
## rename variable names to their labels
#df.rename(columns =  ahsdict.Description.str.replace("\s|:|,|^\d+|\([\w\s\-?]+\)",'').str.lower().to_dict(), inplace=True)
#df.head()

In [16]:
## create a convenience function for variable look up
def varlu(s):
    return(ahsdict[ahsdict.Description.str.lower().str.contains(s)])

### Geographic Data 

Crosswalk data for CBSA to everything else comes from [NBER](http://www.nber.org/data/cbsa-fips-county-crosswalk.html)

In [17]:
#load geographc data
fips= pd.read_csv("/".join([data_path, 'cbsa2fipsxw.csv']), encoding='latin-1')
fips.dropna(how='all',inplace=True)
fips = fips[['cbsacode','statename','cbsatitle','csatitle','countycountyequivalent']]

In [18]:
# CBSA codes do not map uniquely to counties which is a problem 
# because when merging with the main dataset, it matched one to many
# thereby substantially overinflating the number of counties and data
# that are actually in the dataset. 
print("Num of CBSA codes in crosswalk file", fips.cbsacode.shape[0])
print("Num of unique CBSA codes in crosswalk:", fips.cbsacode.unique().shape[0] )

# by dropping duplicatec CBSA codes, I eliminate this issue
# though which county of the many counties mapped to a single CBSA code
# gets mapped into the main dataset is nigh random or arbitrary
fips.drop_duplicates(subset=['cbsacode', 'csatitle'],inplace=True)

Num of CBSA codes in crosswalk file 1882
Num of unique CBSA codes in crosswalk: 929


In [19]:
print("Make sure cbsacode field matched the type in the mother dataframe:")
fips.dtypes

Make sure cbsacode field matched the type in the mother dataframe:


cbsacode                  float64
statename                  object
cbsatitle                  object
csatitle                   object
countycountyequivalent     object
dtype: object

In [20]:
df.OMB13CBSA.dtype

dtype('float64')

In [21]:
## merge with housing data
df = df.merge(fips, left_on = 'OMB13CBSA', right_on = 'cbsacode', how='left')

### drop if we do not know the metro area
df = df.loc[df.cbsatitle.notnull(),:]

print("Size of dataset with CBSAs available")
df.shape

Size of dataset with CBSAs available


(62733, 533)

## Final Scrub

In [22]:
## drop observations for which we do not know renter status
df.dropna(subset=['TENURE'], inplace=True)
print("Size of dataset once observations for which ownership/renter status is not known")
df.shape

Size of dataset once observations for which ownership/renter status is not known


(55834, 533)

In [23]:
## drop EDIT variables
df = df.loc[:,~df.columns.str.contains("^J")]
df.shape

(55834, 234)

In [24]:
## drop mobile home and trailer observations
df = df.loc[df.BLD != 'Mobile home or trailer']
df = df[df.BLD != 'Boat, RV, van, etc.']

In [25]:
df = df[df.BLD != 'Boat, RV, van, etc.']

In [26]:
print("NUmber of observations in full dataset without a full bath: %d" %(df.BATHROOMS =='noFullBath').sum())
df = df.loc[df.BATHROOMS !='noFullBath'] # drop observations without a full bath

print("How many rental units are there with more than 3 bathrooms in each cbsa?")
df.loc[(df.BATHROOMS == 'More than 3 bathrooms') & (df.TENURE =='Rented'),['cbsatitle','HINCP']].groupby('cbsatitle').size()

df = df.loc[df.BATHROOMS != 'More than 3 bathrooms'] ## drop obvservations with 3 bathrooms

NUmber of observations in full dataset without a full bath: 0
How many rental units are there with more than 3 bathrooms in each cbsa?


In [27]:
## drop observations with 0 incomes and 0 rents
df = df[df.HINCP > 0]
df = df[(df.RENT > 0) & df.RENT.notnull()]

## Make Variables

In [35]:
## convenience function for making dummy variables
def makeVar(newVar,oldVar,entry,data=df):
    df.loc[:,newVar] = (df.loc[:,oldVar] == entry).astype(np.float)
    df.loc[df[oldVar].isnull(), newVar] = np.nan
    
def makeDummies(col,makeTitle=True,df = df):
    dummies = pd.get_dummies(df.loc[:,col].replace(-6,np.nan))
    dummies =dummies.loc[:,[x for x in dummies.columns if x is not np.nan]]
    if makeTitle:
        dummies.columns = [col+s for s in dummies.columns.str.replace("-|\s|,|:|\.","")]
    if makeTitle is False:
        dummies.columns = dummies.columns.str.replace("-|\s|,|:|\.","")
    dummies.loc[df[col].isnull()]  = np.nan
    #(dummies==0).sum()/dummies.shape[0]
    return(dummies)

In [29]:
df.loc[:,'logincome'] = np.log(df.HINCP+1)
df.loc[:,'incomeSqrd'] = df.HINCP**2
df.loc[:,'hhincomeper1000'] = df.HINCP/1e3
df.loc[:,'hhagesqrd'] = df.HHAGE**2
df.loc[:,'logRent']= np.log(df.RENT)
df.loc[:,'lengthTenure'] = 2015 - df.HHMOVE
df.loc[:,'collegeOrMore'] = df.HHGRAD.isin(['Professional School Degree (For Ex: MD, DDS, DVM, LLB, JD)','Doctorate Degree (For Ex: PhD, EdD)',
                                           'Master\x92s Degree (For Ex: MA, MS, Meng, Med, MSW, MBA)','Bachelor\x92s degree (For Ex: BA, AB, BS)'])
df.loc[:,'collegeOrMore'] = df.collegeOrMore.astype(np.int32)
df['HHRACE'].replace(" Only| / | ","",regex=True,inplace=True)
df.loc[:,'femaleHH'] = (df.HHSEX == 'Female').astype(np.float)
df.loc[:,'marriedHH'] = (df.HHMAR == 'Married, spouse present').astype(np.float)
df.loc[:,'badSchools'] = (df.NHQSCHOOL == 'Disagree').astype(np.float)
df.loc[:,'lotCrime'] = (df.NHQSCRIME == 'Agree').astype(np.float);
makeVar('secured','ENTRYSYS','Yes')
makeVar('bars','WINBARS','Yes')
makeVar('washer','WASHER','Yes')
makeVar('fridge','FRIDGE','Yes')
makeVar('feltCold','COLD','Yes')
makeVar('notoilet','NOTOIL','Yes')

In [30]:
## make a dummy on weather the unit is professionally managed
df.loc[:,'profMngd'] = df.MGRONSITE.isin(['2-4 unit building has owner on-site',
       '5 or more unit building has owner and/or manager on-site',
       '2-4 unit building has manager on-site'])
df.loc[df.MGRONSITE.isnull(),'profMngd'] = np.nan ## mark NAs

## make dummy for roaches
df.loc[:,'roaches'] = df.ROACH.isin(['Seen daily in the last 12 months',
       'Seen a few times in the last 12 months',
       'Seen weekly in the last 12 months',
       'Seen monthly in the last 12 months'])
df.loc[df.ROACH.isnull(),'roaches'] = np.nan

## make dummy for roadents
df.loc[:,'rodents'] = df.RODENT.isin(['Seen daily in the last 12 months',
       'Seen a few times in the last 12 months',
       'Seen weekly in the last 12 months',
       'Seen monthly in the last 12 months'])
df.loc[df.RODENT.isnull(),'rodents'] = np.nan

In [31]:
## make race variables
df = pd.concat([df, pd.get_dummies(df.ADEQUACY)[['Moderately Inadequate', 'Severely Inadequate']]],axis=1)
df.rename(columns = {"Moderately Inadequate":'moderatelyInadequate','Severely Inadequate':'severelyInadequate'},inplace=True)

In [32]:
## make race variables
df = pd.concat([df, pd.get_dummies(df.HHRACE)[['Black','White','Asian']]],axis=1)
print("Share of sample that is either black, white, or asian: %.2f" %(df[['Black','White','Asian']].sum()/df.shape[0]).sum())

df.loc[:,'OtherRace'] = ((df.Black == 0) & (df.White==0) & (df.Asian == 0)) #.astype(np.float)

Share of sample that is either black, white, or asian: 0.97


In [33]:
## to remove outliers and the like, take a broad swipe 
## exclude observations below and above the 10th and 90th percentiles respectively for rent and incomes

rentPercentiles = [.025,.05,.1,.8,.9,.975]
rentBounds = df.loc[:,['cbsatitle', 'RENT']].dropna().groupby('cbsatitle').quantile(rentPercentiles).unstack(level=1)
rentBounds.columns = rentBounds.columns.droplevel(level=0)
rentBounds.columns = ["rent"+str(s) for s in rentBounds.columns]

incPercentiles = [.05,.1,.8,.9]
incBounds = df.loc[:,['cbsatitle', 'HINCP']].dropna().groupby('cbsatitle').quantile(incPercentiles).unstack(level=1)
incBounds.columns = incBounds.columns.droplevel(level=0)
incBounds.columns = ["inc"+str(s) for s in incBounds.columns]
df = df.merge(pd.concat([incBounds, rentBounds], axis=1), right_index=True, left_on = 'cbsatitle')

In [36]:
## dummies:

## fix some variables before dumping them into creation of dummies
df.loc[:,'BATHROOMS'] = df.BATHROOMS.str.replace("No full bath:[\s\w,?]+",'noFullBath')
### add columns out of which I would like to make dummies
toDummiesList = ['BATHROOMS','BLD','YRBUILT','GARAGE',
                 'PORCH','UNITSIZE']
### make the dummies
dummylist = []
for i,col in enumerate(toDummiesList):
    #print(col)
    dummylist.append(makeDummies(col,df=df))

dummies =  pd.concat(dummylist, axis=1)

In [37]:
dummies.drop(['GARAGENo','PORCHNo'],axis=1,inplace=True)
dummies.columns = dummies.columns.str.replace("squarefeet",'sqft')

In [38]:
df = pd.concat([df,dummies],axis=1)

In [39]:
df = df.loc[df.TENURE != 'Occupied without payment of rent'] ## exclude units with odd status

df.loc[:,'renter'] = (df.TENURE == 'Rented').astype(int)
df.loc[:,'owner']  = (df.TENURE == 'Owned or being bought by someone in your household').astype(int)

print("Is tenure dummy perfect?")
df.renter.sum() + df.owner.sum() == df.shape[0]

Is tenure dummy perfect?


True

## Data Dump

In [40]:
#df.to_csv("/".join([data_path, 'AHS2015prepped.csv']))
df.to_csv("/".join([data_path, 'AHS2015prepped_ver2.csv']), index=False)